# Semantic Kernel + Hugging Faces Embedding

This example shows how to configure Semantic Kernel with Hugging Faces.

It is based on the [Example 06 of memory and embeddings](https://github.com/microsoft/semantic-kernel/blob/22d12cbb5a9c9c002ba0ea79f9511fc119bffb84/samples/notebooks/dotnet/06-memory-and-embeddings.ipynb) of Semantic Kernel

**Step 1**: Install Semantic Kernel 

In [2]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.13.277.1-preview"

Installed Packages Microsoft.SemanticKernel, 0.13.277.1-preview

**Step 2**: Instantiate the Kernel

In [4]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.HuggingFace.TextEmbedding;

var kernelBuilder = new KernelBuilder();

// Store in memory and use local transformer
kernelBuilder.WithMemory(
    new SemanticTextMemory(
        new VolatileMemoryStore(),
        new HuggingFaceTextEmbeddingGeneration(
            new Uri("http://localhost:7860/semantic-kernel"),
            "sentence-transformers/all-MiniLM-L6-v2")
    )
);

var kernel = kernelBuilder.Build();

### Manually Adding Memories

Creating some memories into the volatile store by using `SaveInformationAsync`:


In [5]:
const string MemoryCollectionName = "aboutMe";

await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andrea");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I currently work as a tourist operator");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I currently live in Seattle and have been living there since 2005");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "My family is from New York");


Searching the memory:

In [15]:
var questions = new[]
{
    "what is my name?",
    "where do I live?",
    "I live in",
    "where is my family from?",
    "where have I travelled?",
    "what do I do for work?",
};

foreach (var q in questions)
{
    var response = await kernel.Memory.SearchAsync(MemoryCollectionName, q, minRelevanceScore: 0.5).FirstOrDefaultAsync();
    if (response == null)
    {
        Console.WriteLine(q + " " + "Not found");
    }
    else
    {
        Console.WriteLine(q + " " + response?.Metadata.Text);
    }
}

what is my name? My name is Andrea
where do I live? Not found
I live in Not found
where is my family from? My family is from New York
where have I travelled? Not found
what do I do for work? Not found


I've been trying with different parameters of `minRelevanceScore` but with ADA I get better results.